2019/2020 - MDS-1 - Extracción, Transformación y Carga - Etl

Raquel Fort Serra

# Histograma puntos visitante

Se podrá utilizar toda la información que se encuentra en el campus. 

El fichero de datos sobre el que se trabajará es el de partidosLigaNBA.txt.

A cada una de las preguntas hay que responder explicando brevemente que se pretende hacer antes de lanzar el código.

Al documento lo llamareís con vuestro nombre y apellido. Debeís enviarlo a mi correo de CUNEF antes del final del examen.

El lenguaje para trabajar con Spark podrá ser python o R indistintamente.

## Primera pregunta: Describe brevemente que diferencia el persists, cache y collect en spark. Explica brevemente casos en los que es interesante su aplicación

Los tres son conceptos clave para programar de manera más eficiente con Apache Spark.
Por una parte, el almacenamiento en caché o la persistencia de Spark RDD son técnicas de optimización para aplicaciones de Spark iterativas e interactivas.
El almacenamiento en caché y la persistencia ayudan a almacenar resultados parciales intermedios en la memoria o en un almacenamiento más sólido como el disco para que puedan reutilizarse en las etapas posteriores. 
Hay varios beneficios del almacenamiento en caché RDD y el mecanismo de persistencia en Spark. Funcionan bien con grandes conjuntos de datos, no hay que almacenar en caché el dataFrame con una pequeña cantidad de datos.
- Cost efficient: no es necesario calcular el resultado en cada etapa.
- Time efficient: la computación será más rápida
- Ayuda a reducir el tiempo de ejecución de algoritmos iterativos en conjuntos de datos muy grandes.

Por su parte, la función collect intentará copiar cada elemento del RDD en el programa del driver y, si nuestro RDD es pesado, a continuación se quedará sin memoria y se bloqueará. Para que no suceda, esto debemos intentar reducir el número de elementos del RDD con take o takesample, realizando filtrados o muestreando.


#### CACHE 
Almacena los resultados intermedios solo en MEMORIA. Es decir, el almacenamiento predeterminado del caché RDD es la memoria. El caché RDD simplemente persiste con el nivel de almacenamiento predeterminado MEMORY_ONLY. 

#### PERSISTS
Debido a la muy pequeña y sintáctica diferencia entre el almacenamiento en caché y los métodos de persistencia de los RDD, los dos métodos a menudo se usan indistintamente.
En el método persist (), podemos usar varios niveles de almacenamiento como MEMORY_ONLY, MEMORY_AND_DISK, MEMORY_ONLY_SER, MEMORY_AND_DISK_SER, DISK_ONLY.


#### COLLECT
La acción collect () es la operación más simple y común que devuelve todo el contenido de los RDD al programa del controlador. La aplicación de collect () es una prueba unitaria en la que se espera que todo el RDD encaje en la memoria. Como resultado, facilita la comparación del resultado de RDD con el resultado esperado.
Action Collect () tenía la restricción de que todos los datos deberían caber en la máquina y las copias en el controlador.

## Segunda pregunta: Explica brevemente los pasos a seguir para realizar la carga de un conjunto de datos (pasos que se siguieron en la práctica con datos de logs)

- El primer paso que debemos realizar al momento de cualquier análisis que hagamos es, obviamente, tener los datos con los que vamos a trabajar, por lo que necesitamos importarlos, para ello vamos a utilizar la librería de Pandas de Python. 
- Para realizar este proceso debemos tomar en cuenta el formato y la ubicación del archivo. El formato se refiere a la forma en que codifican los datos. Por lo general, podemos distinguir diferentes esquemas de codificación mirando el final del nombre del archivo. Algunas codificaciones comunes son csv, json, xlsx, entre otros. Por su parte la ubicación del archivo o ruta nos dice dónde están almacenados los datos. Por lo general, se almacenan en la computadora que estamos usando o en internet.
- Como dijimos anteriormente vamos a utilizar la librería de Pandas, por lo que primero que debemos hacer es importarla con el siguiente comando: import pandas as pd. Realizado esto podemos utilizar todos los métodos dentro de esta librería.
- Seguidamente definimos una variable en donde colocamos la ruta en donde se encuentran los datos. 
- Y a continuación podemos utilizar los diferentes métodos de Pandas que se utilizan para leer archivos según el formato. Por ejemplo si el formato que tenmos es csv podemos utilizar df = pd.read_csv("nombrearchivo"). Otros comandos por ejemplo: pd.read_json(), pd.read_excel(), pd.read_sql(), etc.
- Con esta ya todos nuestros datos se encuentran almacenados dentro de la variable df.

## Tercera Pregunta: Índica un tipo de problema que puede empeorar los datos. (pe. Que no exista un representante del CDO en todas las áreas de negocio), pon algún ejemplo específico (pe. Datos duplicados) y cómo lo tratarías con técnicas de data cleaning.

Los conjuntos de datos que podemos obtener no siempre cuentan con el formato o completitud necesarios para ser analizados apropiadamente. Por ejemplo, algo que puede empeorar los datos es que algunas filas tienen valores faltantes. Para solucionarlo 
El proceso de limpieza consiste en eliminar o reemplazar elementos de un conjunto de datos de forma que afecten lo menos posible los resultados finales.
Podemos ver los datos nulos de una columna con el siguiente comando: df.filter("columna is null").show
Para solucionar esto, cuando tenemos muchos valores nulos, tenemos diferentes opciones. 
- Podemos eliminarlos: join_Df1= Name.filter(Name.col(FirstName).isnotnull()).show()
- Si creemos que la existencia de valores nulos para una observación puede aportarnos información, podemos sustituir por el número 0 los valores nulos y por 1 los que no lo son con el siguiente comando: df.withColumn("id", when($"id".isNull, 0).otherwise(1)).show 


## Cuarta tarea: Inicializar spark context y cargar los datos desde el fichero.

In [114]:
from pyspark import SparkContext
sc = SparkContext("local", "First App")

In [115]:
data = "partidosLigaNBA.csv"
data = sc.textFile(data)

In [116]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

#### Parsear

In [117]:
data = data.filter(lambda x: "PTS" not in x).map(lambda x: x.split(":"))

In [118]:
data = data.filter(lambda x: "Playoffs" not in x)

#### EDA

In [119]:
data.take(1) #vemos que forma tiene nuestro dataset

[['Tue, Oct 30, 2007',
  '"7',
  '30 pm"',
  'Utah Jazz',
  '117',
  'Golden State Warriors',
  '96']]

In [120]:
data.count() #y vemos las observaciones con las que cuenta nuestra base de datos

12897

## Quinta tarea: Media de la diferencia de puntos por año

Primero creamos datarfames con las fechas, puntos de los visitantes y puntos de casa por separado. 

In [121]:
fecha = data.map(lambda x: x[0][-4:])
ptosVisit = data.map(lambda x : int(x[4]))
ptosCasa = data.map(lambda x : int(x[6]))
                    

In [122]:
ptosCasa.take(3)

[96, 93, 106]

Ahora agrupamos por año todos los puntos 

In [123]:
ptosVisitFecha = data.map(lambda x: (x[0][-4:], int(x[4])))
ptosVisitFecha = ptosVisitFecha.reduceByKey(lambda x, y: x + y)

In [124]:
ptosVisitFecha.take(2)

[('2017', 84102), ('2013', 128963)]

In [125]:
ptosCasaFecha = data.map(lambda x: (x[0][-a:], int(x[6])))
ptosCasaFecha = ptosCasaFecha.reduceByKey(lambda x, y: x + y)

Ahora creamos otro objeto con la diferencia de puntos entre los visitantes y los de casa por año (hacemos la resta en valores absolutos porque vamos a restar sin saber que puntos son mayores y para que no nos de negativo). 

In [126]:
ptosFecha = data.map(lambda x: (x[0][-4:], abs(int(x[4])-int(x[6]))))
ptosFecha = ptosFecha.reduceByKey(lambda x, y: x + y)
ptosFecha.take(5)

[('2017', 9172),
 ('2013', 14659),
 ('2016', 15397),
 ('2011', 9435),
 ('2014', 14547)]

Ahora vamos a contar el número de partidos por año 

In [127]:
partidosFecha = data.map(lambda x: (x[0][-4:], 1))
partidosFecha = partidosFecha.reduceByKey(lambda x, y: x + y)
partidosFecha.take(3)

[('2017', 803), ('2013', 1324), ('2016', 1333)]

Ahora unimos con join la diferencia de puntos por año y el numero de partidos por fecha 

In [128]:
ptosPartidos = ptosFecha.join(partidosFecha)
ptosPartidos.take(3)

[('2017', (9172, 803)), ('2013', (14659, 1324)), ('2016', (15397, 1333))]

Ahora calculamos la media de la diferencia de puntos por año y utilizamos sortBy para ordenarlos de forma ascendente ya que los datos nos salian desordenados

In [129]:
media = ptosPartidos.map(lambda x: (x[0], int(x[1][0])/int(x[1][1]))).sortBy(lambda x: x[0])
media.collect()

[('2007', 11.096491228070175),
 ('2008', 11.543543543543544),
 ('2009', 11.090425531914894),
 ('2010', 10.86903860711582),
 ('2011', 10.661016949152541),
 ('2012', 10.845318860244234),
 ('2013', 11.071752265861027),
 ('2014', 10.9047976011994),
 ('2015', 11.159969673995452),
 ('2016', 11.550637659414853),
 ('2017', 11.422166874221668)]

## Sexta tarea: ¿Han judado todos los equipos el mismo número de partidos? ¿ Si es qué no a que puede deberse?

En primer lugar, hacemos un map que nos recorra la columna del equipo, por una parte el visitante y por otra el de casa, para saber cuantos partidos ha jugado cada equipo

In [130]:
partidosVisit = data.map(lambda x: (x[3],1))
partidosVisit = partidosVisit.reduceByKey(lambda x, y: x + y)
partidosVisit.take(3)

[('New Orleans Pelicans', 166),
 ('Sacramento Kings', 402),
 ('Charlotte Hornets', 127)]

In [131]:
partidosCasa = data.map(lambda x: (x[5],1))
partidosCasa = partidosCasa.reduceByKey(lambda x, y: x + y)

Unimos las dos bases de datos y sumamos el numero de partidos jugados en casa y como visitante

In [132]:
partidosVC = partidosVisit.join(partidosCasa)
partidosVC.take(3)

[('New Orleans Pelicans', (166, 166)),
 ('Charlotte Hornets', (127, 126)),
 ('Phoenix Suns', (413, 412))]

In [133]:
partidosTotal = partidosVC.map(lambda x: (x[0], (int(x[1][0]) + int(x[1][1])))).sortBy(lambda x: x[1])
partidosTotal.collect() #hemos ordenado de forma ascendente los partidos (con sortBy) 

[('Seattle SuperSonics', 82),
 ('Charlotte Hornets', 253),
 ('New Orleans Pelicans', 332),
 ('New Jersey Nets', 394),
 ('Brooklyn Nets', 435),
 ('New Orleans Hornets', 499),
 ('Charlotte Bobcats', 566),
 ('Sacramento Kings', 804),
 ('Minnesota Timberwolves', 804),
 ('Oklahoma City Thunder', 818),
 ('Phoenix Suns', 825),
 ('New York Knicks', 825),
 ('Milwaukee Bucks', 827),
 ('Detroit Pistons', 829),
 ('Philadelphia 76ers', 834),
 ('Washington Wizards', 844),
 ('Utah Jazz', 846),
 ('Denver Nuggets', 848),
 ('Toronto Raptors', 850),
 ('Portland Trail Blazers', 853),
 ('Los Angeles Clippers', 861),
 ('Orlando Magic', 863),
 ('Dallas Mavericks', 867),
 ('Memphis Grizzlies', 867),
 ('Indiana Pacers', 868),
 ('Houston Rockets', 868),
 ('Chicago Bulls', 873),
 ('Golden State Warriors', 885),
 ('Atlanta Hawks', 896),
 ('Los Angeles Lakers', 897),
 ('Cleveland Cavaliers', 901),
 ('Miami Heat', 917),
 ('Boston Celtics', 930),
 ('San Antonio Spurs', 933)]

Como se puede comprobar, no han jugado todos los equipos el mismo número de partidos, esto se puede explicar porque los partidos de Playoffs son partidos de una competición eliminatoria y solo los equipos clasificados se enfrentan en las finales. 

## Séptima pregunta: ¿Cuantos partidos ha ganado en Enero Clevelant?

Recorremos la columna de la fecha pero solo el mes, que es lo que nos interesa para ver cómo se escribe el mes de enero, y seleccionamos todos los datos que ocurrieron solo en enero

In [134]:
data.take(1)

[['Tue, Oct 30, 2007',
  '"7',
  '30 pm"',
  'Utah Jazz',
  '117',
  'Golden State Warriors',
  '96']]

In [135]:
data.map(lambda x: x[0][5:8]).distinct().collect()

['Apr', 'Jan', 'Nov', 'Jun', 'Mar', 'Oct', 'Dec', 'May', 'Feb']

In [136]:
enero = data.filter(lambda x: x[0][5:8] == 'Jan')

In [137]:
enero.take(1)

[['Wed, Jan 2, 2008',
  '"7',
  '30 pm"',
  'Houston Rockets',
  '93',
  'Boston Celtics',
  '97']]

Hacemos lo mismo con el equipo, recorriendo los equipos visitantes por una parte y los equipos que juegan en casa por otra. Pero nos quedamos en cada caso solo con el número de partidos gnados. Y luego los sumamos para tener el total de partidos ganados

In [138]:
data.map(lambda x: x[3]).distinct().collect()

['New Orleans Pelicans',
 'Sacramento Kings',
 'Charlotte Hornets',
 'Phoenix Suns',
 'San Antonio Spurs',
 'Miami Heat',
 'Golden State Warriors',
 'Brooklyn Nets',
 'Milwaukee Bucks',
 'New Jersey Nets',
 'Seattle SuperSonics',
 'Utah Jazz',
 'Los Angeles Lakers',
 'Indiana Pacers',
 'Houston Rockets',
 'Dallas Mavericks',
 'Oklahoma City Thunder',
 'Washington Wizards',
 'Cleveland Cavaliers',
 'Chicago Bulls',
 'Toronto Raptors',
 'Portland Trail Blazers',
 'New Orleans Hornets',
 'Memphis Grizzlies',
 'Denver Nuggets',
 'Philadelphia 76ers',
 'New York Knicks',
 'Orlando Magic',
 'Boston Celtics',
 'Atlanta Hawks',
 'Detroit Pistons',
 'Minnesota Timberwolves',
 'Charlotte Bobcats',
 'Los Angeles Clippers']

In [139]:
clevVisit = enero.filter(lambda x: x[3] == 'Cleveland Cavaliers')  
clevVisit.take(1)

[['Sun, Jan 6, 2008',
  '"1',
  '00 pm"',
  'Cleveland Cavaliers',
  '93',
  'Toronto Raptors',
  '90']]

In [140]:
ganadosVisit = clevVisit.filter(lambda x: int(x[4]) > int(x[6]))
ganadosVisitSum = ganadosVisit.count()
ganadosVisitSum

41

In [141]:
clevCasa = enero.filter(lambda x: x[5] == 'Cleveland Cavaliers') 
clevCasa.take(1)

[['Wed, Jan 2, 2008',
  '"7',
  '00 pm"',
  'Atlanta Hawks',
  '94',
  'Cleveland Cavaliers',
  '98']]

In [142]:
ganadosCasa = clevCasa.filter(lambda x: int(x[6]) > int(x[4]))
ganadosCasaSum = ganadosCasa.count()
ganadosCasaSum

42

ganadosTotal nos va a dar la respuesta a la séptima pregunta

In [143]:
ganadosTotal = ganadosVisitSum + ganadosCasaSum
ganadosTotal

83

Como podemos observar, los Cleveland han ganado 83 partidos en enero.

## Octava pregunta: ¿Los Warrios son mejores fuera de casa o en casa?

Seleccionamos por una parte los partidos que han jugado los Warriors como visitantes y nos quedamos solo con los ganados

In [144]:
warriorsVisit = data.filter(lambda x: x[3] == 'Golden State Warriors')
warriorsVisit.take(1)

[['Fri, Nov 2, 2007',
  '"7',
  '30 pm"',
  'Golden State Warriors',
  '114',
  'Los Angeles Clippers',
  '120']]

In [145]:
ganadosVisitw = warriorsVisit.filter(lambda x: int(x[4]) > int(x[6]))
ganadosVisitWSum = ganadosVisitw.count()
ganadosVisitWSum

215

Hacemos lo mismo pero con los partidos jugados en casa por los Warriors

In [146]:
warriorsCasa = data.filter(lambda x: x[5] == 'Golden State Warriors')
warriorsCasa.take(1)

[['Tue, Oct 30, 2007',
  '"7',
  '30 pm"',
  'Utah Jazz',
  '117',
  'Golden State Warriors',
  '96']]

In [147]:
ganadosCasaw = warriorsCasa.filter(lambda x: int(x[6]) > int(x[4]))
ganadosCasaWSum = ganadosCasaw.count()
ganadosCasaWSum

308

In [148]:
ganadosVisitWSum, ganadosCasaWSum

(215, 308)

Observamos que hay mayor número de partidos ganados en casa (308) frente a los partidos ganados como visitantes (215)

## Novena pregunta: Equipo que ha quedado primerio en victorias más temporadas. (si es que hay alguno que más)

Obtenemos los distintos años que tenemos, para ver las temporadas

In [149]:
data.map(lambda x: x[0][-4:]).distinct().collect()

['2017',
 '2013',
 '2016',
 '2011',
 '2014',
 '2008',
 '2007',
 '2015',
 '2009',
 '2010',
 '2012']

Seleccionamos los partidos ganados(visita y casa)

In [150]:
ganadosVisit = data.filter(lambda x: int(x[4]) > int(x[6]))
ganadosCasa = data.filter(lambda x: int(x[6]) > int(x[4]))

Vamos a ir buscando los equipos que han ganado más partidos por temporada (años).

2007

In [151]:
g2007visit = ganadosVisit.filter(lambda x: x[0][-4:] == '2007') #seleccionmos de las observaciones ganadas las del año 2007 para visita y para casa  
g2007casa = ganadosCasa.filter(lambda x: x[0][-4:] == '2007')

In [152]:
visita2007 = g2007visit.map(lambda x: (x[3], 1)) 
visita2007 = visita2007.reduceByKey(lambda x, y: x + y) #obtenemos las veces ganadas por equipo visitante
casa2007 = g2007casa.map(lambda x: (x[5], 1))
casa2007 = casa2007.reduceByKey(lambda x, y: x + y) #veces ganadas por equipo que juega en casa
total2007 = visita2007.join(casa2007) 
total2007.take(3)

[('Minnesota Timberwolves', (1, 3)),
 ('Utah Jazz', (6, 11)),
 ('Phoenix Suns', (13, 9))]

In [153]:
final2007 = total2007.map(lambda x: (x[0], (int(x[1][0]) + int(x[1][1])))) #sumamos partido ganados en casa y como visitante por equipos
final2007.max(lambda x: (x[1])) #asi obtenemos el maximo 

('Boston Celtics', 26)

2008

In [154]:
g2008visit = ganadosVisit.filter(lambda x: x[0][-4:] == '2008')
g2008casa = ganadosCasa.filter(lambda x: x[0][-4:] == '2008')

In [155]:
visita2008 = g2008visit.map(lambda x: (x[3], 1))
visita2008 = visita2008.reduceByKey(lambda x, y: x + y)
casa2008 = g2008casa.map(lambda x: (x[5], 1))
casa2008 = casa2008.reduceByKey(lambda x, y: x + y)
total2008 = visita2008.join(casa2008)
total2008.take(3)

[('Atlanta Hawks', (14, 32)),
 ('Utah Jazz', (20, 42)),
 ('Phoenix Suns', (21, 31))]

In [156]:
final2008 = total2008.map(lambda x: (x[0], (int(x[1][0]) + int(x[1][1]))))
final2008.max(lambda x: (x[1]))

('Boston Celtics', 84)

2009

In [160]:
g2009visit = ganadosVisit.filter(lambda x: x[0][-4:] == '2009')
g2009casa = ganadosCasa.filter(lambda x: x[0][-4:] == '2009')
visita2009 = g2009visit.map(lambda x: (x[3], 1))
visita2009 = visita2009.reduceByKey(lambda x, y: x + y)
casa2009 = g2009casa.map(lambda x: (x[5], 1))
casa2009 = casa2009.reduceByKey(lambda x, y: x + y)
total2009 = visita2009.join(casa2009)
final2009 = total2009.map(lambda x: (x[0], (int(x[1][0]) + int(x[1][1]))))
final2009.max(lambda x: (x[1]))

('Los Angeles Lakers', 81)

2010

In [161]:
g2010visit = ganadosVisit.filter(lambda x: x[0][-4:] == '2010')
g2010casa = ganadosCasa.filter(lambda x: x[0][-4:] == '2010')
visita2010 = g2010visit.map(lambda x: (x[3], 1))
visita2010 = visita2010.reduceByKey(lambda x, y: x + y)
casa2010 = g2010casa.map(lambda x: (x[5], 1))
casa2010 = casa2010.reduceByKey(lambda x, y: x + y)
total2010 = visita2010.join(casa2010)
final2010 = total2010.map(lambda x: (x[0], (int(x[1][0]) + int(x[1][1]))))
final2010.max(lambda x: (x[1]))

('Los Angeles Lakers', 71)

2011

In [163]:
g2011visit = ganadosVisit.filter(lambda x: x[0][-4:] == '2011')
g2011casa = ganadosCasa.filter(lambda x: x[0][-4:] == '2011')
visita2011 = g2011visit.map(lambda x: (x[3], 1))
visita2011 = visita2011.reduceByKey(lambda x, y: x + y)
casa2011 = g2011casa.map(lambda x: (x[5], 1))
casa2011 = casa2011.reduceByKey(lambda x, y: x + y)
total2011 = visita2011.join(casa2011)
final2011 = total2011.map(lambda x: (x[0], (int(x[1][0]) + int(x[1][1]))))
final2011.max(lambda x: (x[1]))

('Chicago Bulls', 53)

2012

In [164]:
g2012visit = ganadosVisit.filter(lambda x: x[0][-4:] == '2012')
g2012casa = ganadosCasa.filter(lambda x: x[0][-4:] == '2012')
visita2012 = g2012visit.map(lambda x: (x[3], 1))
visita2012 = visita2012.reduceByKey(lambda x, y: x + y)
casa2012 = g2012casa.map(lambda x: (x[5], 1))
casa2012 = casa2012.reduceByKey(lambda x, y: x + y)
total2012 = visita2012.join(casa2012)
final2012 = total2012.map(lambda x: (x[0], (int(x[1][0]) + int(x[1][1]))))
final2012.max(lambda x: (x[1]))

('San Antonio Spurs', 82)

2013

In [165]:
g2013visit = ganadosVisit.filter(lambda x: x[0][-4:] == '2013')
g2013casa = ganadosCasa.filter(lambda x: x[0][-4:] == '2013')
visita2013 = g2013visit.map(lambda x: (x[3], 1))
visita2013 = visita2013.reduceByKey(lambda x, y: x + y)
casa2013 = g2013casa.map(lambda x: (x[5], 1))
casa2013 = casa2013.reduceByKey(lambda x, y: x + y)
total2013 = visita2013.join(casa2013)
final2013 = total2013.map(lambda x: (x[0], (int(x[1][0]) + int(x[1][1]))))
final2013.max(lambda x: (x[1]))

('Miami Heat', 85)

2014

In [166]:
g2014visit = ganadosVisit.filter(lambda x: x[0][-4:] == '2014')
g2014casa = ganadosCasa.filter(lambda x: x[0][-4:] == '2014')
visita2014 = g2014visit.map(lambda x: (x[3], 1))
visita2014 = visita2014.reduceByKey(lambda x, y: x + y)
casa2014 = g2014casa.map(lambda x: (x[5], 1))
casa2014 = casa2014.reduceByKey(lambda x, y: x + y)
total2014 = visita2014.join(casa2014)
final2014 = total2014.map(lambda x: (x[0], (int(x[1][0]) + int(x[1][1]))))
final2014.max(lambda x: (x[1]))

('San Antonio Spurs', 73)

2015

In [167]:
g2015visit = ganadosVisit.filter(lambda x: x[0][-4:] == '2015')
g2015casa = ganadosCasa.filter(lambda x: x[0][-4:] == '2015')
visita2015 = g2015visit.map(lambda x: (x[3], 1))
visita2015 = visita2015.reduceByKey(lambda x, y: x + y)
casa2015 = g2015casa.map(lambda x: (x[5], 1))
casa2015 = casa2015.reduceByKey(lambda x, y: x + y)
total2015 = visita2015.join(casa2015)
final2015 = total2015.map(lambda x: (x[0], (int(x[1][0]) + int(x[1][1]))))
final2015.max(lambda x: (x[1]))

('Golden State Warriors', 88)

2016

In [168]:
g2016visit = ganadosVisit.filter(lambda x: x[0][-4:] == '2016')
g2016casa = ganadosCasa.filter(lambda x: x[0][-4:] == '2016')
visita2016 = g2016visit.map(lambda x: (x[3], 1))
visita2016 = visita2016.reduceByKey(lambda x, y: x + y)
casa2016 = g2016casa.map(lambda x: (x[5], 1))
casa2016 = casa2016.reduceByKey(lambda x, y: x + y)
total2016 = visita2016.join(casa2016)
final2016 = total2016.map(lambda x: (x[0], (int(x[1][0]) + int(x[1][1]))))
final2016.max(lambda x: (x[1]))

('Golden State Warriors', 87)

2017

In [169]:
g2017visit = ganadosVisit.filter(lambda x: x[0][-4:] == '2017')
g2017casa = ganadosCasa.filter(lambda x: x[0][-4:] == '2017')
visita2017 = g2017visit.map(lambda x: (x[3], 1))
visita2017 = visita2017.reduceByKey(lambda x, y: x + y)
casa2017 = g2017casa.map(lambda x: (x[5], 1))
casa2017 = casa2017.reduceByKey(lambda x, y: x + y)
total2017 = visita2017.join(casa2017)
final2017 = total2017.map(lambda x: (x[0], (int(x[1][0]) + int(x[1][1]))))
final2017.max(lambda x: (x[1]))

('Golden State Warriors', 54)

##  Décima pregunta: Escribe la expresión regular correcta que sólo macheen los teléfonos y el correo del siguiente texto.

Si eres cliente y necesitas información sobre tus posiciones, productos o realizar operaciones: Desde España. Desde el extranjero. Banca telefónica en castellano. Bandera castellano. 902 13 23 13. Banca telefónica en catalán. Bandera catalana. 902 88 30 08. Banca telefónica en inglés. Bandera inglesa. 902 88 88 35. O por correo electrónico a atencioncliente@bankinter.com

In [100]:
import re

In [101]:
texto = "Si eres cliente y necesitas información sobre tus posiciones, productos o realizar operaciones: Desde España. Desde el extranjero. Banca telefónica en castellano. Bandera castellano. 902 13 23 13. Banca telefónica en catalán. Bandera catalana. 902 88 30 08. Banca telefónica en inglés. Bandera inglesa. 902 88 88 35. O por correo electrónico a atencioncliente@bankinter.com"

In [102]:
tlf = '\d{3} \d{2} \d{2} \d{2}'
mail = '\S+[@]\S+[.com]'

In [103]:
re.findall(tlf, texto)

['902 13 23 13', '902 88 30 08', '902 88 88 35']

In [104]:
re.findall(mail, texto)

['atencioncliente@bankinter.com']

In [111]:
sc.stop()